In [ ]:
n_qubits = 3

## Step 1: Create Grover n-qubit, compile to L3, and save ISA (QPY)


In [ ]:
from __future__ import annotations

from datetime import datetime

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit import qpy

from ibm_quantum_connector import QuantumServiceManager
from grover_algorithm import grover_algorithm

# Connect and select backend
svc = QuantumServiceManager(config_file='quantum_config.json')
assert svc.connect(), 'Failed to connect to IBM Quantum service'
backend = svc.select_backend()
assert backend is not None, 'Failed to select backend'

# Build source circuit and compile to L3
src_qc = grover_algorithm(n_qubits)
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
isa = pm.run(src_qc)

# Save ISA QPY
from pathlib import Path
artifacts = Path('artifacts')
artifacts.mkdir(exist_ok=True)

ts = datetime.now().strftime('%Y%m%d_%H%M%S')
qpy_path = artifacts / f'grover{n_qubits}_l3_{backend.name}_{ts}.qpy'
with open(qpy_path, 'wb') as f:
    qpy.dump([isa], f)

print('Saved ISA QPY to:', qpy_path)
isa


In [ ]:
# Draw the L3 ISA circuit and save a PNG next to the QPY
import matplotlib.pyplot as plt
from pathlib import Path

try:
    fig = isa.draw(output='mpl')
    png_path = Path(str(qpy_path).replace('.qpy', '.png'))
    fig.savefig(png_path, dpi=180, bbox_inches='tight')
    print('Saved circuit PNG to:', png_path)
    plt.show()
except NameError as e:
    raise RuntimeError('Please run Step 1 cell first to create `isa` and `qpy_path`.') from e


In [ ]:
from qiskit import qpy
import glob

# Find the latest qpy file in the artifacts directory
qpy_files = sorted(glob.glob('artifacts/*.qpy'), reverse=True)
if not qpy_files:
    raise FileNotFoundError("No .qpy files found in the artifacts directory.")
qpy_path = qpy_files[0]

with open(qpy_path, 'rb') as f:
    isa = list(qpy.load(f))[0]


In [ ]:
from quantum_executor import QuantumExecutor
qe = QuantumExecutor( config_file='quantum_config.json')

qe.run_circuit(isa_circuit=isa, shots=1024, execution_type='ideal_simulator')

In [ ]:
qe.run_circuit(isa_circuit=isa, shots=1024, execution_type='noisy_simulator')

In [ ]:
qe.run_circuit(isa_circuit=isa, shots=8192, execution_type='real_device')

## Step 2

In [ ]:
from quantum_executor import QuantumExecutor
from delta_debug import run_delta_debug_on_isa

# Step 2: Run Delta Debugging on the compiled ISA circuit `isa`
marked_states = [2**n_qubits - 1]

qe = QuantumExecutor(config_file='quantum_config.json')
result = run_delta_debug_on_isa(
    executor=qe,
    isa_circuit=isa,
    n_qubits=n_qubits,
    marked_states=marked_states,
    tolerance=0.1,
    test_mode=False
)


In [ ]:
# ========== Generate and display the HTML visualization report ==========
from delta_debug_visualizer import generate_html_report
from IPython.display import IFrame, display

# Generate the HTML report
html_path = generate_html_report(result)

# Display key information in English
print(f"\n📊 Key Findings:")
print(f"  • Total segments: {result['total_segments']}")
print(f"  • Problematic segments: {len(result['problematic_segments'])}")
print(f"  • Problematic segment IDs: {result['problematic_segments']}")
print(f"  • Baseline loss: {result['baseline_loss']:.4f}")
print(f"  • Test count: {result['test_count']}")

print(f"\n✅ HTML report generated: {html_path}")
print("👇 See the interactive report below")

# Display the report directly in the notebook
display(IFrame(html_path, width=1000, height=800))